In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [15]:
mv = pd.read_csv('tmdb_5000_movies.csv')
cr = pd.read_csv('tmdb_5000_credits.csv')
df = mv.merge(cr , on='title')

In [16]:
df = df[['title' , 'genres' , 'id' , 'keywords' , 'overview' ,  'cast' , 'crew']]
df.dropna(inplace=True)
df.duplicated().sum()

0

In [17]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def top_3_cast(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
        if(len(L)==3):
            break
    return L

def get_director(obj):
    for i in ast.literal_eval(obj):
        if(i['job'] == 'Director'):
            return i['name']


In [18]:
df['genres'] = df['genres'].apply(convert)
df['keywords'] = df['keywords'].apply(convert)
df['cast'] = df['cast'].apply(top_3_cast)
df['director'] = df['crew'].apply(get_director)
df['overview'] = df['overview'].apply(lambda x : x.split())
df.drop(['crew'] , axis = 1 , inplace = True)
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.drop(['index'] , inplace=True , axis = 1)

In [19]:
df['genres'] = df['genres'].apply(lambda x: [i.replace(" " , "") for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [i.replace(" " , "") for i in x])
df['cast'] = df['cast'].apply(lambda x: [i.replace(" " , "") for i in x])

df['director'] = df['director'].apply(lambda x: x.replace(' ' , ''))
df['director'] = df['director'].apply(lambda x : x.split())

In [20]:
df['tags'] = df['keywords'] + df['overview'] + df['cast'] + df['director']
df = df[['id' , 'title' , 'tags']]

In [21]:
df['tags'] = df['tags'].apply(lambda x : " ".join(x))
df['tags'] = df['tags'].apply(lambda x : x.lower())

In [22]:
from nltk.stem.porter import  PorterStemmer

ps = PorterStemmer()

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

df['tags'] = df['tags'].apply(stem)

In [23]:
from sklearn.feature_extraction.text import  CountVectorizer
cv = CountVectorizer(max_features = 5000 , stop_words = 'english')
vectors = cv.fit_transform(df['tags']).toarray()

In [2]:
from sklearn.metrics.pairwise import  cosine_similarity
similarity = cosine_similarity(vectors)

100.0


In [64]:
pd.options.mode.chained_assignment = None

df[['similar']] = ''
for i in range(df.shape[0]):
    top_5 = sorted(list(enumerate(similarity[i])) , reverse=True , key = lambda x:x[1])[1:6]
    temp = []
    for j in top_5:
        ind = j[0]
        temp.append([df['title'][ind] , df['id'][ind]])
    df['similar'][i] = temp
        


In [68]:
df

,id,title,tags,1,2,3,4,5,similar
0,19995,Avatar,cultureclash futur spacewar spacecoloni societ...,Aliens vs Predator: Requiem,Aliens,Battle: Los Angeles,Independence Day,Falcon Rising,"[[Aliens vs Predator: Requiem, 440], [Aliens, ..."
1,285,Pirates of the Caribbean: At World's End,ocean drugabus exoticisland eastindiatradingco...,Pirates of the Caribbean: Dead Man's Chest,Pirates of the Caribbean: The Curse of the Bla...,"20,000 Leagues Under the Sea",Pirates of the Caribbean: On Stranger Tides,The Pirates! In an Adventure with Scientists!,"[[Pirates of the Caribbean: Dead Man's Chest, ..."
2,206647,Spectre,spi basedonnovel secretag sequel mi6 britishse...,Skyfall,Never Say Never Again,Quantum of Solace,Safe Haven,From Russia with Love,"[[Skyfall, 37724], [Never Say Never Again, 366..."
3,49026,The Dark Knight Rises,dccomic crimefight terrorist secretident burgl...,The Dark Knight,Batman Returns,Batman,Batman Forever,Batman,"[[The Dark Knight, 155], [Batman Returns, 364]..."
4,49529,John Carter,basedonnovel mar medallion spacetravel princes...,Riddick,Get Carter,Krrish,The Other Side of Heaven,The Peanuts Movie,"[[Riddick, 87421], [Get Carter, 10461], [Krris..."
...,...,...,...,...,...,...,...,...,...
4771,9367,El Mariachi,unitedstates–mexicobarri leg arm paperknif gui...,The R.M.,Should've Been Romeo,2:13,Copying Beethoven,Interview with the Assassin,"[[The R.M., 36825], [Should've Been Romeo, 113..."
4772,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...,Purple Violets,She's the One,Our Family Wedding,A Perfect Getaway,One Missed Call,"[[Purple Violets, 20065], [She's the One, 1136..."
4773,231617,"Signed, Sealed, Delivered",date loveatfirstsight narrat investig team pos...,Team America: World Police,Eden,Miracle,Narc,The Game of Their Lives,"[[Team America: World Police, 3989], [Eden, 36..."
4774,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...,Killers,Should've Been Romeo,I Am Sam,I Served the King of England,Take Me Home Tonight,"[[Killers, 37821], [Should've Been Romeo, 1134..."


In [67]:
import pickle

In [66]:
pickle.dump(df , open('movies.pkl' , 'wb'))

In [ ]:
a